In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT =  os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ["LANGCHAIN_ENDPONT"] =  "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = LANGCHAIN_PROJECT

In [4]:
LANGCHAIN_PROJECT

'Langgraph end to end'

In [5]:
#reading model from google
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
   
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")


/home/brian/Documents/JIM/Langgraph/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name = "gemma2-9b-it")

### Simple AI Assistant

In [7]:
while True:
    question = input("Type your question. If you want to quit the chat write quit.")
    if question != "quit":
        print(llm.invoke(question).content)
    else:
        print("Goodbye take care")
        break

Hello! 👋  How can I help you today? 😄

Hello! 👋  How can I help you today? 😄

It seems like you're asking me to be quiet. 

Is there anything else I can help you with?  😊 

Goodbye take care


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

config = {"configurable": {"session_id": "firstchat"}}
model_with_memory = RunnableWithMessageHistory(llm, get_session_history)

In [10]:
model_with_memory.invoke(("Hi, I'm Jim"), config=config).content

"Hi Jim, it's nice to meet you! What can I do for you today?\n"

In [11]:
model_with_memory.invoke(("tell me my what is my name ?"), config=config).content

'You told me your name is Jim! 😊  \n\nIs there anything else I can help you with?\n'

In [12]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi, I'm Jim", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Jim, it's nice to meet you! What can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37, 'completion_time': 0.04, 'prompt_time': 0.00187667, 'queue_time': 0.2378481, 'total_time': 0.04187667}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a5c91b89-6f27-45af-8fda-185e3b167010-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37}), HumanMessage(content='tell me my what is my name ?', additional_kwargs={}, response_metadata={}), AIMessage(content='You told me your name is Jim! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 53, 'total_toke

### RAG with LCEL

In [13]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [14]:
# reading the text files from source directory
loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

#creating chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [15]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [16]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [17]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Llama 3 is a large language model. Here are 3 important points:

* **8B parameter version:** There is a version of Llama 3 with 8 billion parameters. 
* **Released in April 2024:**  Llama 3 was released to the public in April 2024.
* **Accessible outside:** Versions of Llama 3 are available for use outside of Meta's internal systems. 


Let me know if you have any other questions! 



### Tools And Agents

In [18]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper = api_wrapper)

In [21]:
tool.name

'wikipedia'

In [22]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [23]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [26]:
print(tool.run({"query": "LangGraph"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: List of unsolved problems in mathematics
Summary: Many mathematical problems have been stated but not yet solved. These problems come from many areas of mathematics, such as theoretical physics, computer science, algebra, analysis, combinatorics, algebraic, differential, discrete and Euclidean geometries, graph theory, group theory, model theory, number theory, set theory, Ramsey theory, dynamical systems, and partial differential equations. Some problems belong to more than one discipline and are studied using techniques from different areas. Prizes are often awarded for the solution to a long-standing problem, and some lists of unsolv

In [27]:
from langchain_community.tools import YouTubeSearchTool
youtube_tool = YouTubeSearchTool()

In [28]:
youtube_tool.name

'youtube_search'

In [30]:
youtube_tool.run("Krish Naik")

"['https://www.youtube.com/watch?v=JxgmHe2NyeY&pp=ygUKS3Jpc2ggTmFpaw%3D%3D', 'https://www.youtube.com/watch?v=p4pHsuEf4Ms&pp=ygUKS3Jpc2ggTmFpaw%3D%3D']"

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool = TavilySearchResults()

/tmp/ipykernel_67572/706410539.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults()


In [ ]:
tavily_tool.invoke({"query": "What happened in the Israel Iran conflict ?"})

[{'title': 'Israel-Iran conflict: A timeline of how the attacks unfolded',
  'url': 'https://www.independent.co.uk/news/world/middle-east/israel-iran-conflict-timeline-attacks-b2770765.html',
  'content': 'In a major escalation, Israel announced on Friday morning that around 200 Israeli Air Force aircraft dropped 330 munitions on around 100 targets in total overnight, hitting six cities including Tehran and Iran’s nuclear facility in Natanz.\n\nAtleast 20 senior commanders were killed, two regional sources told _Reuters_, including the chief of staff of Iran’s armed forces, and senior leaders in the Revolutionary Guards. [...] Overnight on Saturday, Israel targeted Iran’s defence ministry headquarters in Tehran, while footage showed a blaze raging at the nearby Shahran oil depot.\n\nAround a million people were forced into air raid shelters as sirens were heard in Tel Aviv and Jerusalem. Nearly 200 were wounded and six killed in a strike on an eight-storey building in Bat Yam, a city s